In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "round summary" in dict.keys():
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 2. TURN: 1 


You confidently raise your hand, announcing a bid of $150. The auctioneer nods in your direction, acknowledging your bid. The crowd murmurs among themselves, some nodding in approval while others seem hesitant, scanning the visible items in the storage room for anything that might justify a higher bid. 

The auctioneer calls out, "We have $150! Do I hear more?" 

Your heart races as you wait for the next bid, hoping your gamble pays off.

possible actions:
1. Increase your bid to assert dominance.
2. Wait and see if anyone outbids you.
3. Try to get a better look at the items in the storage from where you stand, to see if there's anything of particular value.

I select option 3. TURN: 2 


skill: spot_hidden
difficulty: medium

I am making a skill check using SPOT_HIDDEN against a difficulty of MEDIUM.
And I rolled a 75 for a result of FAILURE. TURN: 3 


From where you stand, you strain your eyes to make out the details of the items within the storage unit, but the angle and distance make it difficult to discern anything of clear value. The shadows cast by the overhead lights further obscure your view, leaving you uncertain about the potential treasures or junk that might be hidden among the clutter.

As you're trying to get a better look, another bidder, a well-dressed woman standing at the back, raises her hand and bids $200. The auctioneer's voice cuts through the air, "We have $200! Can I get $250?"

Your attempt to spot something valuable has not yielded any advantage, and the bid has now increased.

possible actions:
1. Bid $250 to stay in the game.
2. Hold your current position and wait to see if the price goes even higher.
3. Engage in a bit of fast talk to dissuade others from bidding higher.

I select option 1. TURN: 4 


With determination, you step back into the fray, raising your hand to bid $250. The auctioneer's eyes light up as he sees your bid, "We have $250 from the gentleman over here! Do I hear $300?"

Your bid injects a moment of hesitation among the crowd. You can see the calculations running through their minds, weighing the risk and potential reward. The well-dressed woman who previously outbid you narrows her eyes, contemplating her next move.

The auctioneer continues to scan the crowd for any sign of further interest, his voice echoing off the walls of the storage facility.

possible actions:
1. Prepare to increase your bid if challenged.
2. Stand firm with your current bid, hoping no one dares to go higher.
3. Try to read the body language of the other bidders to gauge their interest.

...

I select option 1. TURN: 2 


You decide to start with the boxes, hoping to find something valuable among the mundane. The first box you open seems to contain nothing but old books with worn-out covers, their pages yellowed by time. However, as you dig deeper, you uncover a small, leather-bound journal. Its pages are filled with frantic handwriting, sketches of strange symbols, and mentions of places you've never heard of, possibly related to the Cthulhu Mythos. The journal speaks of rituals, otherworldly beings, and the pursuit of forbidden knowledge. It's unsettling, and you can't shake off the feeling of being watched.

possible actions:
1. Continue reading the journal to uncover more about its contents.
2. Put the journal aside and open another box, hoping to find something less disturbing.
3. Take a break from the boxes and examine the peculiar artifacts instead, to see if they're connected to the journal.

I select option 1. TURN: 3 


skill: sanity
difficulty: medium

I am making a skill check using SANITY against a difficulty of MEDIUM.
And I rolled a 54 for a result of FAILURE. TURN: 4 


As you delve deeper into the journal, the writings become increasingly erratic and desperate. The author describes their descent into madness, driven by an obsession with an entity known as "The Great Dreamer." The sketches of the symbols start to move in your peripheral vision, a trick of the mind surely, but unnerving nonetheless. Your heart races, and the room seems to close in on you. The words blur together, forming messages that seem directed at you, promising knowledge and power at a terrible cost. The air grows colder, and a whispering voice fills the room, though you're alone. The failure to maintain your sanity in the face of these revelations leaves you shaken, questioning the nature of reality itself.

You close the journal, trying to shake off the feeling of dread that has settled over you. The experience has left you deeply unsettled, and you can't 

I select option 3. TURN: 5 


You turn your attention to the peculiar artifacts, hoping to ground yourself in the tangible after the unsettling experience with the journal. Among the artifacts, you find a statuette that immediately draws your eye. It's an odd figure, carved from a dark, almost absorbing material. The figure is neither human nor animal, but a grotesque amalgamation of both, with too many limbs and eyes that seem to follow you as you move. It's unlike anything you've ever seen, and despite your recent scare, you're drawn to it with a mix of fear and fascination.

As you examine the statuette, the air around you grows heavy, and a low hum begins to fill the room, emanating from nowhere and everywhere at once. The sensation of being watched intensifies, and you realize that the artifacts in this room might hold more than just monetary value—they could be objects of significant power, possibly dangerous and certainly mysterious.

Suddenly, from the far corner of the storage

IndexError: list index out of range